#Ноутбук для обучения ms-marco моделей на смержинных текстах

In [1]:
dataset_names = ['initial', '2 same topics', '2 different topics', '3 same topics', '3 different topics']
dataset_paths = ['/content/drive/MyDrive/QA_training/extractive_QA/datasets_csv/ms_initial_train.csv',
                 '/content/drive/MyDrive/QA_training/extractive_QA/merged datasets/ms_marco/merged_by_2_train.csv',
                 '/content/drive/MyDrive/QA_training/extractive_QA/merged datasets/ms_marco/old_merged_datasets/ms_merged_train.csv',
                 '/content/drive/MyDrive/QA_training/extractive_QA/merged datasets/ms_marco/merged_by_3_train.csv',
                 '/content/drive/MyDrive/QA_training/extractive_QA/merged datasets/ms_marco/random_merged_by_3_train.csv']

In [2]:
model_names = ['3 same topic longformer2roberta', '3 different topics longformer2roberta']
model_paths = ['/content/drive/MyDrive/QA_training/generative_QA/ms_marco/models/3_same_topics_longformer2roberta',
               '/content/drive/MyDrive/QA_training/generative_QA/ms_marco/models/3_same_topics_longformer2roberta_epoch_3_4']

# Обучение

In [3]:
!pip install import-ipynb
%cd '/content/drive/MyDrive/QA_training/generative_QA'
import import_ipynb
from support_functions import *

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp37-none-any.whl size=2976 sha256=8a21e72294cf18644e28fcf95ebd5e4410bbf52c00c47da87fa5c3e184184dab
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb
/content/drive/MyDrive/QA_training/generative_QA
importing Jupyter notebook from support_functions.ipynb
     |████████████████████████████████| 2.3MB 8.2MB/s 
     |████████████████████████████████| 3.3MB 68.1MB/s 
     |████████████████████████████████| 901kB 59.2MB/s 
     |████████████████████████████████| 245kB 7.6MB/s 
     |████████████████████████████████| 245kB 11.5MB/s 
     |████████████████████████████████| 122kB 14.4MB/s 


In [4]:
from transformers import LongformerTokenizerFast

tokenizer = LongformerTokenizerFast.from_pretrained("allenai/longformer-base-4096")

In [5]:
def read_data_long(path, text_num=1, parts_num = 1, part = 1, text_ids = None):
    '''
    path - path to csv file,
    parts_num - number of parts that initial dataset is divided for
    part - number of part that wil be selected
    text_num - number of texts that dataset contains
    text_ids - sertain number of texts, that will be used. If none - all text will be used

    '''
    df = pd.read_csv(path)

    i_start = int((part-1) / parts_num * len(df))
    i_finish = int(part / parts_num * len(df))

    questions = df.question[i_start:i_finish].to_list()
    answers = df.answer[i_start:i_finish].fillna('no answer').to_list()

    if text_num != 1:
            if text_ids:
                if len(text_ids) == 1:
                    contexts = df[f"t{text_ids[0]}"][i_start:i_finish].to_list()
                else:
                    text_col_names = [f"t{i}" for i in text_ids]
                    texts = df.loc[i_start:i_finish, text_col_names].to_numpy()
                    contexts = [' '.join(item) for item in texts]
            else:    
                text_col_names = [f"t{i+1}" for i in range(text_num)]
                texts = df.loc[i_start:i_finish, text_col_names].to_numpy()
                contexts = [' '.join(item) for item in texts]
    else:
        contexts = df.text[i_start:i_finish].to_list()

    return contexts, questions, answers

class LongGenerativeDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels


    def __getitem__(self, idx):
        item = {}
        item['input_ids'] = self.encodings.input_ids[idx]
        item['attention_mask'] = self.encodings.attention_mask[idx]
        item['decoder_input_ids'] = self.labels.input_ids[idx]
        item['decoder_attention_mask'] = self.labels.attention_mask[idx]
        item['labels'] = self.labels.input_ids[idx].clone()
        item['global_attention_mask'] = self.encodings.global_attention_mask[idx]
        # because BERT automatically shifts the labels, the labels correspond exactly to `decoder_input_ids`. 
        # We have to make sure that the PAD token is ignored
        item['labels'][item['labels'] == tokenizer.pad_token_id] = -100


        return item

    def __len__(self):
        return len(self.encodings.input_ids)

def prepare_data_long(path, tokenizer, text_num=1, parts_num = 1, part = 1, text_ids = None):
    contexts, questions, answers = read_data_long(path, text_num, parts_num, part, text_ids)
    qa_pairs = [f"Question: {q} Context: {c}" for q, c in zip(questions, contexts)]

    #encodings = tokenizer(questions, contexts, add_special_tokens=True, padding=True, truncation=True, return_tensors='pt', max_length=512)
    encodings = tokenizer(qa_pairs, padding=True, truncation=True, return_tensors='pt', max_length=512)
    labels = tokenizer(answers, padding=True, truncation=True, return_tensors='pt', max_length = 64)
    global_idx = list(map(lambda x: (x==43885).nonzero(as_tuple=True)[0][0], encodings['input_ids']))
    encodings['global_attention_mask'] = torch.ones_like(encodings['input_ids'])
  

    for i, mask in zip(global_idx, encodings['global_attention_mask']):
        mask[:i] = 2

    train_dataset = LongGenerativeDataset(encodings, labels)

    return train_dataset

In [6]:
train_csv_path = dataset_paths[3]
print('Dataset path:', train_csv_path)


train_dataset = prepare_data_long(train_csv_path, tokenizer,  text_num=3, parts_num = 2, part = 1)

Dataset path: /content/drive/MyDrive/QA_training/extractive_QA/merged datasets/ms_marco/merged_by_3_train.csv


In [7]:
dir = model_paths[1]

#print('Importing model path:', model_v4_paths[number-2])
print('New model path:', dir)
'''
model = EncoderDecoderModel.from_encoder_decoder_pretrained("allenai/longformer-base-4096", "roberta-base", 
                                                            bos_token_id=tokenizer.bos_token_id, 
                                                            eos_token_id=tokenizer.eos_token_id, 
                                                            pad_token_id=tokenizer.pad_token_id)

'''
model = EncoderDecoderModel.from_pretrained(model_paths[0])
#model = EncoderDecoderModel.from_pretrained(dir+'/checkpoint-90000')


New model path: /content/drive/MyDrive/QA_training/generative_QA/ms_marco/models/3_same_topics_longformer2roberta_epoch_3_4


In [8]:
training_args = TrainingArguments(
    output_dir=dir,          # output directory
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=4,  # batch size per device during training
    per_device_eval_batch_size=8,   # batch size for evaluation
    logging_dir=dir+'/logs',            # directory for storing logs
    logging_steps=500,
    save_steps = 2000,
    prediction_loss_only=True,
    overwrite_output_dir = True,
    learning_rate = 1e-5
)


trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    #eval_dataset=val_dataset,            # evaluation dataset

)

In [ ]:
print('Dataset path:', train_csv_path)
print('New model path:', dir)

torch.cuda.empty_cache()
trainer.train()
#trainer.train(dir+'/checkpoint-90000')
trainer.save_model(dir)

Dataset path: /content/drive/MyDrive/QA_training/extractive_QA/merged datasets/ms_marco/merged_by_3_train.csv
New model path: /content/drive/MyDrive/QA_training/generative_QA/ms_marco/models/3_same_topics_longformer2roberta_epoch_3_4


Step,Training Loss
52500,1.061800
53000,1.035900
53500,1.035000
54000,0.980000
54500,0.962000
55000,0.961000
55500,1.018200
56000,1.026600
56500,0.957300
57000,0.964300


In [ ]:
logs = model_paths[0] + '/logs'
logs

'/content/drive/MyDrive/QA_training/generative_QA/ms_marco/models/bert2bert/logs'

In [ ]:
!kill 319

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
%tensorboard --inspect --logdir bert2bert:'/content/drive/MyDrive/QA_training/generative_QA/ms_marco/models/bert2bert/logs'

ERROR: Failed to launch TensorBoard (exited with 0).
Contents of stderr:
2021-05-24 08:11:20.090120: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Contents of stdout:
Processing event files... (this can take a few minutes)

No event files found within logdir bert2bert:/content/drive/MyDrive/QA_training/generative_QA/ms_marco/models/bert2bert/logs

In [ ]:
%tensorboard --logdir_spec 3_same_bert2bert:/content/drive/MyDrive/QA_training/generative_QA/ms_marco/models/3_same_topics_bert2bert/logs,3_diff_bert2bert:/content/drive/MyDrive/QA_training/generative_QA/ms_marco/models/3_defferent_topics_bert2bert/logs

In [ ]:
%tensorboard --logdir /content/drive/MyDrive/QA_training/generative_QA/ms_marco/models/3_same_topics_longformer2roberta_attempt_3/logs